In [1]:
import pickle
import pandas as pd
import numpy as np

In [2]:
credit_risk_test = pd.read_csv('home-credit-default-risk/application_test.csv')

In [3]:
unused_columns =[
    "SK_ID_CURR",  #id
    
    "OWN_CAR_AGE", "EXT_SOURCE_1", "APARTMENTS_AVG", "BASEMENTAREA_AVG", "YEARS_BEGINEXPLUATATION_AVG", 
    "YEARS_BUILD_AVG", "COMMONAREA_AVG", "ELEVATORS_AVG", "ENTRANCES_AVG", "FLOORSMAX_AVG", 
    "FLOORSMIN_AVG", "LANDAREA_AVG", "LIVINGAPARTMENTS_AVG", "LIVINGAREA_AVG", "NONLIVINGAPARTMENTS_AVG", 
    "NONLIVINGAREA_AVG", "APARTMENTS_MODE", "BASEMENTAREA_MODE", "YEARS_BEGINEXPLUATATION_MODE", 
    "YEARS_BUILD_MODE", "COMMONAREA_MODE", "ELEVATORS_MODE", "ENTRANCES_MODE","FLOORSMAX_MODE", 
    "FLOORSMIN_MODE", "LANDAREA_MODE", "LIVINGAPARTMENTS_MODE", "LIVINGAREA_MODE", 
    "NONLIVINGAPARTMENTS_MODE", "NONLIVINGAREA_MODE", "APARTMENTS_MEDI", "BASEMENTAREA_MEDI", 
    "YEARS_BEGINEXPLUATATION_MEDI", "YEARS_BUILD_MEDI", "COMMONAREA_MEDI", "ELEVATORS_MEDI", 
    "ENTRANCES_MEDI", "FLOORSMAX_MEDI", "FLOORSMIN_MEDI", "LANDAREA_MEDI", "LIVINGAPARTMENTS_MEDI", 
    "LIVINGAREA_MEDI", "NONLIVINGAPARTMENTS_MEDI", "NONLIVINGAREA_MEDI", "FONDKAPREMONT_MODE",
    "HOUSETYPE_MODE", "TOTALAREA_MODE", "WALLSMATERIAL_MODE", "EMERGENCYSTATE_MODE", "FLAG_MOBIL",
]

In [4]:
test = credit_risk_test.loc[:, credit_risk_test.columns.isin(
    ["SK_ID_CURR"]) + ~credit_risk_test.columns.isin(unused_columns)].reset_index(drop=True)

In [5]:
education_encoder={
    None: 0,
    "Lower secondary": 1,
    "Secondary / secondary special": 2,
    "Incomplete higher": 3,
    "Higher education": 4,
    "Academic degree": 4
}

In [6]:
test["CODE_GENDER"] = test["CODE_GENDER"].replace("XNA", np.nan)
test["NAME_FAMILY_STATUS"] = test["NAME_FAMILY_STATUS"].replace("Unknown", np.nan)
test["OCCUPATION_TYPE"] = test["OCCUPATION_TYPE"].replace("XNA", np.nan)
test["NAME_EDUCATION_TYPE"] = test["NAME_EDUCATION_TYPE"].replace(education_encoder)

In [7]:
drop_columns = [
    "NAME_CONTRACT_TYPE", "CODE_GENDER", "NAME_FAMILY_STATUS", 
    "CNT_FAM_MEMBERS", "DAYS_LAST_PHONE_CHANGE"
]

test.dropna(subset=drop_columns, inplace=True)
test.reset_index(drop=True, inplace=True)

In [8]:
from sklearn.impute import SimpleImputer

In [9]:
imputation_mode = SimpleImputer(strategy='most_frequent')
imputation_median = SimpleImputer(strategy='median')
imputation_mean = SimpleImputer(strategy='mean')

attr_mode = ["NAME_TYPE_SUITE"]
attr_median = ["AMT_ANNUITY", "AMT_GOODS_PRICE", "OBS_30_CNT_SOCIAL_CIRCLE", 
               "DEF_30_CNT_SOCIAL_CIRCLE", "OBS_60_CNT_SOCIAL_CIRCLE", 
               "DEF_60_CNT_SOCIAL_CIRCLE", "AMT_REQ_CREDIT_BUREAU_HOUR", 
               "AMT_REQ_CREDIT_BUREAU_DAY", "AMT_REQ_CREDIT_BUREAU_WEEK", 
               "AMT_REQ_CREDIT_BUREAU_MON", "AMT_REQ_CREDIT_BUREAU_QRT", 
               "AMT_REQ_CREDIT_BUREAU_YEAR"]
attr_mean = ["EXT_SOURCE_2", "EXT_SOURCE_3"]

attr_implementation = {'mode': [imputation_mode, attr_mode],
            'median': [imputation_median, attr_median],
            'mean': [imputation_mean, attr_mean]}

for a in attr_implementation.values():
    for attr in a[1]:
        test[attr] = a[0].fit_transform(test[[attr]])

In [10]:
dummy_data = test.iloc[0]
dummy_data.NAME_INCOME_TYPE = "Maternity leave"
dummy_data.to_frame().T

C:\Users\HP\AppData\Local\Temp\ipykernel_15976\2321654027.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dummy_data.NAME_INCOME_TYPE = "Maternity leave"


,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
test = test.append(dummy_data, ignore_index = True)
test[test.NAME_INCOME_TYPE == "Maternity leave"].index

C:\Users\HP\AppData\Local\Temp\ipykernel_15976\59935975.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test = test.append(dummy_data, ignore_index = True)


Int64Index([48744], dtype='int64')

In [12]:
categorical_columns = [col for col in test.select_dtypes(include='object')]
onehot_test = pd.get_dummies(test[[x for x in categorical_columns if x != "NAME_EDUCATION_TYPE"]], drop_first=True)
onehot_test.drop(48744, inplace=True)
onehot_test.head()

,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_M,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,...,ORGANIZATION_TYPE_Trade: type 4,ORGANIZATION_TYPE_Trade: type 5,ORGANIZATION_TYPE_Trade: type 6,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_Transport: type 1,ORGANIZATION_TYPE_Transport: type 2,ORGANIZATION_TYPE_Transport: type 3,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_University,ORGANIZATION_TYPE_XNA
0,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [13]:
test.drop(48744, inplace=True)

In [14]:
from sklearn.preprocessing import StandardScaler

In [15]:
numerical_columns = [col for col in test.columns.tolist() if col not in categorical_columns + ["SK_ID_CURR"]]
ss = StandardScaler()

std_test = pd.DataFrame(ss.fit_transform(test[numerical_columns]), columns=numerical_columns)
std_test.head()

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_EDUCATION_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,-0.559988,-0.427809,0.142475,-0.553580,-0.037477,1.648953,-0.164654,-0.733477,-0.483656,-0.056958,...,-0.039517,0.0,0.0,0.0,-0.042538,-0.036585,-0.048265,-0.078421,-0.711321,-1.153865
1,-0.559988,-0.782413,-0.804537,-0.752831,-0.839362,-0.611196,1.009586,-0.461392,-0.498482,-1.168264,...,-0.039517,0.0,0.0,0.0,-0.042538,-0.036585,-0.048265,-0.078421,-0.711321,0.589324
2,-0.559988,0.237075,0.401002,2.520066,0.497113,1.648953,-0.147258,-0.917718,-0.498406,0.786092,...,-0.039517,0.0,0.0,0.0,-0.042538,-0.036585,-0.048265,-0.078421,0.773588,1.170387
3,2.260729,1.345214,2.896221,1.223666,3.303709,-0.611196,0.358078,0.483623,-0.480449,0.835352,...,-0.039517,0.0,0.0,0.0,-0.042538,-0.036585,-0.048265,-0.078421,-0.711321,0.589324
4,0.850370,0.015447,0.297651,0.165019,0.483748,-0.611196,-0.775825,0.699997,-0.482700,0.272381,...,-0.039517,0.0,0.0,0.0,-0.042538,-0.036585,-0.048265,-0.078421,-0.711321,0.008261


In [16]:
data_model_test = pd.concat([test["SK_ID_CURR"], onehot_test, std_test], axis=1)

In [17]:
data_model_test.iloc[:,1:]

,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_M,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,0,0,0,1,0,0,0,0,0,1,...,-0.039517,0.0,0.0,0.0,-0.042538,-0.036585,-0.048265,-0.078421,-0.711321,-1.153865
1,0,1,0,1,0,0,0,0,0,1,...,-0.039517,0.0,0.0,0.0,-0.042538,-0.036585,-0.048265,-0.078421,-0.711321,0.589324
2,0,1,1,1,0,0,0,0,0,1,...,-0.039517,0.0,0.0,0.0,-0.042538,-0.036585,-0.048265,-0.078421,0.773588,1.170387
3,0,0,0,1,0,0,0,0,0,1,...,-0.039517,0.0,0.0,0.0,-0.042538,-0.036585,-0.048265,-0.078421,-0.711321,0.589324
4,0,1,1,0,0,0,0,0,0,1,...,-0.039517,0.0,0.0,0.0,-0.042538,-0.036585,-0.048265,-0.078421,-0.711321,0.008261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,0,0,0,1,0,0,0,0,0,1,...,-0.039517,0.0,0.0,0.0,-0.042538,-0.036585,-0.048265,-0.078421,-0.711321,-0.572802
48740,0,0,0,0,0,0,0,0,0,1,...,-0.039517,0.0,0.0,0.0,-0.042538,-0.036585,-0.048265,-0.078421,-0.711321,0.008261
48741,0,0,1,1,0,0,0,0,0,1,...,-0.039517,0.0,0.0,0.0,-0.042538,-0.036585,-0.048265,-0.078421,3.743406,-0.572802
48742,0,1,0,0,1,0,0,0,0,0,...,-0.039517,0.0,0.0,0.0,-0.042538,-0.036585,-0.048265,-0.078421,-0.711321,0.008261


In [18]:
loaded_model = pickle.load(open('credit_risk_prediction.pkl', 'rb'))

In [19]:
test_pred = loaded_model.predict(data_model_test.iloc[:,1:])
test_proba = loaded_model.predict_proba(data_model_test.iloc[:,1:])

C:\Users\HP\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- NAME_EDUCATION_TYPE
Feature names seen at fit time, yet now missing:
- index

  warnings.warn(message, FutureWarning)
C:\Users\HP\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- NAME_EDUCATION_TYPE
Feature names seen at fit time, yet now missing:
- index

  warnings.warn(message, FutureWarning)


In [20]:
test_result = pd.DataFrame({"SK_ID_CURR": data_model_test.iloc[:, 0].tolist(),
                            "PREDICTION": test_pred.tolist(),
                            "PROBABILITY": test_proba[:,1].tolist()})
test_result

,SK_ID_CURR,PREDICTION,PROBABILITY
0,100001,0,0.06
1,100005,0,0.17
2,100013,0,0.09
3,100028,0,0.15
4,100038,0,0.11
...,...,...,...
48739,456221,0,0.02
48740,456222,0,0.24
48741,456223,0,0.08
48742,456224,0,0.08


In [21]:
save_result = test_result[["SK_ID_CURR", "PROBABILITY"]]
save_result.rename({"PROBABILITY": "TARGET"}, axis=1, inplace=True)
save_result

C:\Users\HP\AppData\Local\Temp\ipykernel_15976\3167122453.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  save_result.rename({"PROBABILITY": "TARGET"}, axis=1, inplace=True)


,SK_ID_CURR,TARGET
0,100001,0.06
1,100005,0.17
2,100013,0.09
3,100028,0.15
4,100038,0.11
...,...,...
48739,456221,0.02
48740,456222,0.24
48741,456223,0.08
48742,456224,0.08


In [22]:
save_result.to_csv("result_prediction.csv", index=False)